In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pydataset import data

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression


In [4]:
df = data('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


Load the tips dataset.

a. Create a column named price_per_person. This should be the total bill divided by the party size.

b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

c. Use select k best to select the top 2 features for predicting tip amount. What are they?

d. Use recursive feature elimination to select the top 2 features for tip amount. What are they?
e. Why do you think select k best and recursive feature elimination might give different answers for the top features?    Does this change as you change the number of features your are selecting?

In [5]:
df['price_per_person'] = df.total_bill / df['size']


In [6]:
X = df[df.columns.drop('tip').tolist()]
y = df.tip
X.head()

,total_bill,sex,smoker,day,time,size,price_per_person
1,16.99,Female,No,Sun,Dinner,2,8.495000
2,10.34,Male,No,Sun,Dinner,3,3.446667
3,21.01,Male,No,Sun,Dinner,3,7.003333
4,23.68,Male,No,Sun,Dinner,2,11.840000
5,24.59,Female,No,Sun,Dinner,4,6.147500


In [7]:
dummies_list = X.select_dtypes(object).columns

dummy_df = pd.get_dummies(X[dummies_list], drop_first=True)
encoded = pd.concat([X, dummy_df], axis = 1)

X = encoded.drop(columns = dummies_list)
X.head()

,total_bill,size,price_per_person,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
1,16.99,2,8.495000,0,0,0,1,0,0
2,10.34,3,3.446667,1,0,0,1,0,0
3,21.01,3,7.003333,1,0,0,1,0,0
4,23.68,2,11.840000,1,0,0,1,0,0
5,24.59,4,6.147500,0,0,0,1,0,0


In [9]:
kb_selector = SelectKBf_selector = SelectKBest(f_regression, k=2)
kb_selector.fit(X, y)
feature_mask = kb_selector.get_support()
f_feature = X.iloc[:,feature_mask].columns.tolist()
f_feature

['total_bill', 'size']

In [10]:
lm = LinearRegression()

rfe = RFE(lm, n_features_to_select=2)

rfe.fit(X,y)  

feature_mask = rfe.support_

rfe_feature = X.iloc[:,feature_mask].columns.tolist()
rfe_feature

['total_bill', 'smoker_Yes']

In [11]:
from sklearn.preprocessing import MinMaxScaler

columns_to_scale = ['total_bill', 'size', 'price_per_person']

scaler = MinMaxScaler()
X_scaled = X.copy()

In [12]:
X_scaled[columns_to_scale] = pd.DataFrame(scaler.fit_transform(X[columns_to_scale]), 
                                                  columns=X[columns_to_scale].columns.values).set_index([X.index.values])

X_scaled.head()


,total_bill,size,price_per_person,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
1,0.291579,0.2,0.322989,0,0,0,1,0,0
2,0.152283,0.4,0.032854,1,0,0,1,0,0
3,0.375786,0.4,0.237261,1,0,0,1,0,0
4,0.431713,0.2,0.515230,1,0,0,1,0,0
5,0.450775,0.6,0.188075,0,0,0,1,0,0


In [13]:
lm = LinearRegression()

rfe = RFE(lm, n_features_to_select=2)

rfe.fit(X_scaled,y)  

feature_mask = rfe.support_

rfe_feature = X_scaled.iloc[:,feature_mask].columns.tolist()
rfe_feature

['total_bill', 'price_per_person']

Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [14]:
def select_kbest(X, y, number_of_features_to_select = 3):
    f_selector = SelectKBest(f_regression, k=number_of_features_to_select)
    f_selector.fit(X, y)
    feature_mask = f_selector.get_support()
    f_feature = X.iloc[:,feature_mask].columns.tolist()
    return f_feature

In [15]:
select_kbest(X, y, number_of_features_to_select = 2)

['total_bill', 'size']

Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [16]:
def rfe_using_linear_regression(X, y, number_of_features_to_select = 3):
    lm = LinearRegression()

    rfe = RFE(lm, n_features_to_select=number_of_features_to_select)

    rfe.fit(X,y)  

    feature_mask = rfe.support_

    rfe_feature = X.iloc[:,feature_mask].columns.tolist()
    return rfe_feature

In [17]:
rfe_using_linear_regression(X_scaled, y, number_of_features_to_select=2)


['total_bill', 'price_per_person']

In [18]:
swiss = data('swiss')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [20]:
swiss_X = swiss.drop(columns='Fertility')
swiss_X.head()

,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,17.0,15,12,9.96,22.2
Delemont,45.1,6,9,84.84,22.2
Franches-Mnt,39.7,5,5,93.40,20.2
Moutier,36.5,12,7,33.77,20.3
Neuveville,43.5,17,15,5.16,20.6


In [21]:
swiss_y = swiss.Fertility
swiss_y.head()

Courtelary      80.2
Delemont        83.1
Franches-Mnt    92.5
Moutier         85.8
Neuveville      76.9
Name: Fertility, dtype: float64

In [22]:
columns_to_scale = ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality']

scaler = MinMaxScaler()
swiss_X_scaled = swiss_X.copy()

In [23]:
swiss_X_scaled[columns_to_scale] = pd.DataFrame(scaler.fit_transform(swiss_X[columns_to_scale]), 
                                                  columns=swiss_X[columns_to_scale].columns.values).set_index([swiss_X.index.values])

swiss_X_scaled.head()


,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,0.178531,0.352941,0.211538,0.079816,0.721519
Delemont,0.496045,0.088235,0.153846,0.845069,0.721519
Franches-Mnt,0.435028,0.058824,0.076923,0.932550,0.594937
Moutier,0.398870,0.264706,0.115385,0.323148,0.601266
Neuveville,0.477966,0.411765,0.269231,0.030761,0.620253


In [24]:
select_kbest(swiss_X_scaled, swiss_y)


['Examination', 'Education', 'Catholic']